# **Project Title: Face Mask Detection using TensorFlow**

**Dataset link:-**
/kaggle/input/face-mask-detection

/kaggle/input/face-mask-12k-images-dataset

# **Objective:**
**To build a deep learning model that automatically detects whether people in images are wearing face masks or not using object detection techniques.**

# **Dataset Description:**

**Images of people in public places with/without face masksAnnotations provided in Pascal VOC XML format, or converted to CSV format with:filename, xmin, ymin, xmax, ymax, classLabels:with_mask (or class 0)without_mask (or class 1)**



# **Workflow Steps:**
**Setup Environment**

Install TensorFlow Object Detection API and dependencies

Configure directory structure

**Prepare Data**

Load dataset and convert annotations (XML/CSV → TFRecord)

Create label map (label_map.pbtxt)

**Model Selection**

Choose a pre-trained model

Modify pipeline.config for the dataset

**Train the Model**

Train using model_main_tf2.py

Save checkpoints, logs

**Evaluate the Model**

Visualize loss and accuracy using TensorBoard

Export trained model

**Run Inference**

Load the trained model

Predict on test images

Visualize bounding boxes with predicted classes and confidence

# **Cell 1: Library Imports**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import PIL

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# **Cell 2: Preparing directories**


In [ ]:
# Path
img_dir = '/kaggle/input/face-mask-detection/images'
"""
annotation_directory contains files, and each file is associated to only one image,
and it contains the height and width of the image and also xmin,ymin,xmax, and ymax of each boundary box
inside the image
"""
annotation_dir = '/kaggle/input/face-mask-detection/annotations'

input_dir = '/kaggle/input/face-mask-detection'
output_dir = '/kaggle/working/'

# **Cell 3: Viewing some images**



In [ ]:
# sample visuaization
for idx, image in enumerate(os.listdir(img_dir)):
    img = cv2.imread(os.path.join(img_dir, image), 1)
    plt.imshow(img)
    plt.show()
    
    if idx == 3:
        break

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    boxes = []
    filename = root.find('filename').text

    for obj in root.findall('object'):
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append([filename, xmin, ymin, xmax, ymax, label])

    return boxes

annotation_list = []

for xml_file in os.listdir(annotation_dir):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(annotation_dir, xml_file)
        annotation_list.extend(parse_annotation(xml_path))

df = pd.DataFrame(annotation_list, columns=["filename", "xmin", "ymin", "xmax", "ymax", "label"])
df.head()


# **Cell 4: Storing images paths**


In [ ]:
img_file_path = []
for img in os.listdir(img_dir): # img here is the name of the image not the image itself
    image = cv2.imread(os.path.join(img_dir,img), 0) # 0 for grayscale
    img_file_path.append(f'{img}')

# **Cell 5: Libraries needed for reading XML files**




In [ ]:
import xml.etree.ElementTree as ET 
"""
ElementTree module provides 
a way to work with Extensible Markup Language (XML) documents as a tree-like structure of elements.
"""
import glob

# **Cell 6: Reading XML files**

In [ ]:
df = { 'name': [],
        'label': [],
      'width': [],
      'height': [],
     'xmin': [],
     'ymin': [],
     'xmax': [],
     'ymax': []}
"""
The glob.glob() function returns a list of all the pathnames that match the specified pattern.
The resulting list can then be used to process or analyze the files that match the pattern.
"""
for idx, anno in enumerate(glob.glob(annotation_dir + '/*.xml')):
    trees = ET.parse(anno)
    
    #print(anno) print/view the annotation to understand the following code
    root = trees.getroot()
    width, height = [], []
    for item in root.iter():
        if item.tag == 'size':
            for attr in list(item):
                if attr.tag == 'width':
                    width = int(round(float(attr.text)))
                if attr.tag == 'height':
                    height = int(round(float(attr.text)))
                    
        if item.tag == 'object':
            for attr in list(item):
                if 'name' in attr.tag:
                    label = attr.text
                    df['label'] += [label]
                    df['width'] += [width]
                    df['height'] += [height]
                    #dataset['name']+=[anno.split('/')[-1][0:-4]] 
                    df['name'] += [anno.split('/')[-1][0:-4]]
                    
                if 'bndbox' in attr.tag:
                    for dim in attr:
                        if dim.tag == 'xmin':
                            xmin = int(round(float(dim.text)))
                            df['xmin'] += [xmin]
                            
                        if dim.tag == 'ymin':
                            ymin = int(round(float(dim.text)))
                            df['ymin'] += [ymin]
                        if dim.tag == 'xmax':
                            xmax = int(round(float(dim.text)))
                            df['xmax'] += [xmax]
                        if dim.tag == 'ymax':
                            ymax = int(round(float(dim.text)))
                            df['ymax'] += [ymax]
                    

# **Cell 7: Viewing data frame of images**


In [ ]:
df1 = pd.DataFrame(df)
df1.head()

# **Cell 8: Describing data frame**

In [ ]:
df1.info()

# **Cell 9: Maping labels to integers**


In [ ]:
label_map = { 'without_mask': 0,
            'with_mask': 1,
            'mask_weared_incorrect': 2}

df1['class'] = df1['label'].map(label_map)

In [ ]:
df1.head()

# **Cell 10: Splitting the data**

In [ ]:
# split train, test, val data
from sklearn.model_selection import train_test_split

train, test = train_test_split(img_file_path, test_size=0.2, random_state=101)
train, val = train_test_split(train, test_size=0.15, random_state=101)

# **Cell 11: Preparing yolo v5 model.**

In [ ]:
# yolo v5
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -qr requirements.txt

# **Cell 12: Preparing some new directories.**

In [ ]:

import os

# Change the working directory
os.chdir('/kaggle/working')

# Define base path for YOLOv5 data
base_path = './yolov5/data'

# List of subdirectories to create
subdirs = [
    'train/images',
    'train/labels',
    'val/images',
    'val/labels',
    'test/images',
    'test/labels'
]

# Create each subdirectory
for subdir in subdirs:
    full_path = os.path.join(base_path, subdir)
    os.makedirs(full_path, exist_ok=True)  # Avoids FileExistsError


# **Cell 13: Copying the image data in the yolov5 folder**

In [ ]:
import os
from PIL import Image

# Correct path to the images
img_dir = '/kaggle/input/face-mask-detection/images'
output_dir = '/kaggle/working'

def open_image_file(image_items, folder_name):
    for image in image_items:
        try:
            img_path = os.path.join(img_dir, image)
            img = Image.open(img_path)
            img1 = img.resize((640, 480))
            save_path = os.path.join(output_dir, f'yolov5/data/{folder_name}/images/{image}')
            img1.save(save_path)
        except Exception as e:
            print(f"Error processing {image}: {e}")




# **Cell 14: Resizing boxes to match with the new images size**

In [ ]:
df1['xmin'] = (640/df1['width']) * df1['xmin']
df1['ymin'] = (480/df1['height']) * df1['ymin']
df1['xmax'] = (640/df1['width']) * df1['xmax']
df1['ymax'] = (480/df1['height']) * df1['ymax']
df1.head()

In [ ]:
df1[['xmin', 'ymin', 'xmax', 'ymax']] = df1[['xmin', 'ymin', 'xmax', 'ymax']].astype('int')

In [ ]:
WIDTH = 640
HEIGHT = 480

# **Cell 15: Converting from left upper corner and right bottom corner of boxes to just midpoint, height, and width of boxes**

In [ ]:
df1['x_center'] = (df1['xmin']+df1['xmax'])/(2*WIDTH)
df1['y_center'] = (df1['ymin']+df1['ymax'])/(2*HEIGHT)
df1['box_width'] = (df1['xmax']-df1['xmin'])/ WIDTH
df1['box_height'] = (df1['ymax']-df1['ymin'])/ HEIGHT

In [ ]:
df1.head()

In [ ]:
df1 = df1.astype(str)

# **Cell 16: Viewing some images paths**

In [ ]:
img_file_path[:5]

# **Cell 17: Writing info for each images**

In [ ]:
def copy_label(label_items, folder_name):
    file_name = [x.split('.')[0] for x in img_file_path]
    for name in file_name:
        data = df1[df1.name == name]
        box_list = []
        for idx in range(len(data)):
            row = data.iloc[idx]
            box_list.append(row['class']+" "+row['x_center']+" "+row['y_center']+" "+ row['box_width']+" "+row['box_height'])

        text = "\n".join(box_list)
        with open(f'{output_dir}/yolov5/data/{folder_name}/labels/{name}.txt', 'w') as file:
            file.write(text)

In [ ]:
copy_label(train, 'train')
copy_label(val, 'val')
copy_label(test, 'test')

In [ ]:
os.chdir('/kaggle/working/yolov5/data/train/labels')

In [ ]:
cat maksssksksss0.txt 

# **Cell 18: Creating yaml file**

In [ ]:
# Configure .yaml file 
yaml_file = """train: /kaggle/working/yolov5/data/train/images
val: /kaggle/working/yolov5/data/val/images
                
nc: 3
names: [without_mask, with_mask, mask_weared_incorrect]"""

with open('/kaggle/working/yolov5/data/data.yaml', 'w') as f:
    f.write(yaml_file)

In [ ]:
%cat /kaggle/working/yolov5/data/data.yaml

# **Cell 19: Training**

In [ ]:
!python /kaggle/working/yolov5/train.py --img 640 --epochs 100 --batch 32 --data /kaggle/working/yolov5/data/data.yaml  --weights yolov5s.pt  --cache

# **Cell 20: Displaying performance of model**

In [ ]:
from IPython.display import Image as Display

In [ ]:
from IPython.display import display, Image

img_path = '/kaggle/working/yolov5/runs/train/exp/labels.jpg'
display(Image(filename=img_path, width=1080))



In [ ]:
img_path = '/kaggle/working/yolov5/runs/train/exp/labels_correlogram.jpg'
display(Image(filename=img_path, width=1080))


# **Cell 21: Displaying Training and Validation batches**

In [ ]:
Display(filename='/kaggle/working/yolov5/runs/train/exp/train_batch0.jpg', width=600)

In [ ]:
from PIL import Image
from IPython.display import display

img = Image.open("/kaggle/working/yolov5/runs/train/exp/train_batch0.jpg")
display(img)


# **Final Summary**
***This project implements an end-to-end Face Mask Detection system using the TensorFlow Object Detection API. The dataset comprises real-world images of people in public with and without face masks. After preparing annotations and converting them into TFRecords, a pre-trained model was fine-tuned to detect masks on faces with high accuracy. The final model was capable of detecting multiple faces and classifying them as "with mask", "without mask", or optionally "mask worn incorrectly". The system can help enforce public safety compliance and can be integrated into surveillance systems or crowd-monitoring tools.***